# Collection

## Scraper

In [1]:
# scaper
from mechanicalsoup import Browser
from bs4 import BeautifulSoup
import re
import urllib
import pandas as pd
import numpy as np

def get_table_by_number(soup, number):
    tables = soup.findAll("table")
    data = []
    for row in tables[number].findAll('tr')[0:]:
        cols = row.findAll('td')
        row = []
        for col in cols:
            row.append(col)
        data.append(row)
    return data

def save_to_csv(url, pageStart, pageEnd, fileName):
    url += ';size=200'
    matrix = []
    for page_no in range(pageStart, pageEnd + 1):
        #print("info from:", url + ";page=" + str(page_no))
        page = urllib.urlopen(url + ";page=" + str(page_no))
        html = page.read()
        soup = BeautifulSoup(html)
        data = get_table_by_number(soup, 2)[1:]
        for row in data:
            matrix_row = []
            for r in row:
                text = r.getText().encode("ascii")
                if len(text) != 0:
                    matrix_row.append(text)
            if len(matrix) == 0:
                matrix = np.array(matrix_row)
            else:
                matrix = np.vstack([matrix, np.array(matrix_row)])

        pd.DataFrame(matrix).to_csv(fileName, sep=',')


def save_to_csv_grounds(url, url2, pageStart, pageEnd, fileName):
    matrix = []
    for page_no in range(pageStart, pageEnd + 1):
        page = urllib.request.urlopen(url + str(page_no) + url2)
        html = page.read()
        soup = BeautifulSoup(html)
        data = get_table_by_number(soup, 2)[1:]
        for i in range(0, len(data)-1, 2):
            row = data[i]
            row2 = data[i+1]
            #print(row2)
            matrix_row = [row[0].getText().encode("ascii"),
                          int(row2[6].getText().encode("ascii"))/ (int(row2[2].getText().encode("ascii"))*2)]
            if len(matrix) == 0:
                matrix = np.array(matrix_row)
            else:
                matrix = np.vstack([matrix, np.array(matrix_row)])
        pd.DataFrame(matrix).to_csv(fileName, sep=',')

# url = 'http://stats.espncricinfo.com/ci/engine/stats/index.html?class=2;filter=advanced;orderby=team_score;size=200
# ;spanmin1=05+Jan+1994;spanval1=span;template=results;type=team;view=innings' saveToCsv(url, 1, 30,
# "Match_Details.csv")

url = 'http://stats.espncricinfo.com/ci/engine/stats/index.html?class=2;page='
url2 = ';spanmin1=13+Jun+2006;spanval1=span;template=results;type=aggregate;view=ground'
save_to_csv_grounds(url, url2, 1, 2, "Ground_Stats.csv")

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


## File_data_cleaner

In [2]:
import pandas as pd
import numpy as np


def clean_innings_for_scenario_1():
    df = pd.read_csv("Data1.csv")
    df_grounds = pd.read_csv("Ground_Stats.csv")
    data = df.as_matrix()
    grounds = df_grounds.as_matrix()        #use .values()  instead of  .as_matrix() in future
    matrix = []
    data = data[data[:, 2] == 215]
    j=0
    
#    print("######################")
#    for i in data:
#        print(data)
#        j=j+1
#        if(j==10):
#            break
#    print("######################")
#    print(grounds)
    count = 0
    for row in data:
        matrix_row = []
        for g in grounds:
            if row[-1] in str(g[1].encode("ascii")):
                matrix_row = row.copy()
                print(matrix_row)
                matrix_row =np.append(matrix_row[1:], [g[-1], 10-int(row[-2]),300-215])
                break
        if len(matrix_row) == 0:
            matrix_row = row.copy()
            matrix_row = np.append(matrix_row[1:], [250, 10-int(row[-2]),300-215])
        if len(matrix) == 0:
            matrix = np.array(matrix_row)
        else:
            matrix = np.vstack([matrix, np.array(matrix_row)])

    pd.DataFrame(matrix).to_csv("second_innings.csv", sep=',')


clean_innings_for_scenario_1()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


[215 1 215 156 4 'Eden Gardens']
[216 1 215 157 4 'Eden Gardens']
[520 2 215 168 4 'County Ground']
[827 3 215 147 4 'Kennington Oval']
[1132 4 215 104 5 'Warner Park, Basseterre']
[1438 5 215 226 2 'Gaddafi Stadium']
[1740 6 215 224 3 'National Stadium']
[2359 8 215 208 3 'Sir Vivian Richards Stadium, North Sound']
[2809 10 215 193 3 'Nehru Stadium']
[2810 10 215 200 3 'Nehru Stadium']
[3115 11 215 201 4 'Sydney Cricket Ground']
[3425 12 215 141 5 'Sir Vivian Richards Stadium, North Sound']
[3731 13 215 177 4 'Trent Bridge']
[4257 15 215 170 6 'Gaddafi Stadium']
[4565 18 215 175 3 'Sheikh Zayed Stadium']
[6034 23 215 209 4 'Sydney Cricket Ground']
[6719 26 215 167 3 'Newlands']
[7021 27 215 168 3 'Melbourne Cricket Ground']
[7631 29 215 192 2 'Sheikh Zayed Stadium']
[8566 32 215 217 2 'Harare Sports Club']
[8877 33 215 192 2 'Sheikh Abu Naser Stadium']
[9797 36 215 155 4 'Sheikh Zayed Stadium']
[10099 37 215 186 1 'Sheikh Zayed Stadium']
[10403 38 215 174 3 'Shere Bangla National Stad

[349041 1310 215 124 6 'Civil Service Cricket Club, Stormont']
[349945 1313 215 126 6 'Shere Bangla National Stadium']
[350506 1315 215 103 8 'Gymkhana Club Ground']
[350762 1316 215 182 1 'SuperSport Park']
[351077 1317 215 132 8 'Kensington Oval, Bridgetown']
[351621 1319 215 104 7 'Shere Bangla National Stadium']
[351881 1320 215 157 5 'Kennington Oval']
[352195 1321 215 129 8 'Shere Bangla National Stadium']
[353361 1325 215 131 7 'Newlands']
[353975 1327 215 236 2 'Captain Roop Singh Stadium']
[354624 1330 215 147 4 'Shere Bangla National Stadium']
[355857 1334 215 183 4 'Sir Vivian Richards Stadium, North Sound']
[356163 1335 215 160 4 'Melbourne Cricket Ground']
[356467 1336 215 151 5 'Shere Bangla National Stadium']
[356759 1337 215 160 7 'Eden Park']
[357351 1339 215 94 6 'Newlands']
[357630 1340 215 215 4 'Kingsmead']
[357934 1341 215 164 4 'Shere Bangla National Stadium']
[358553 1343 215 130 3 'Buffalo Park']
[358554 1343 215 131 3 'Buffalo Park']
[359472 1346 215 172 2 'Qu

In [94]:
## Converter

In [3]:
import pandas as pd
import yaml
import numpy as np
import os

def get_venue_average(venue):
    df_grounds = pd.read_csv("Ground_Stats.csv")
    grounds = df_grounds.as_matrix()
    for g in grounds:
        if venue in g[1]:
            return g[-1]
    else:
        return 250


def get_data():
    matrix_1_test = []
    matrix_2_test = []
    matrix_1_train = []
    matrix_2_train = []
    new_matrix= []
    i = 0
    total_overs = 50
    for filename in os.listdir('/Users/jeevan venkataramana/Desktop/odis'):
        matrix_1 = []
        matrix_2 = []
        i += 1
        print(i)
        with open('/Users/jeevan venkataramana/Desktop/odis/' + filename, 'r') as f:
            doc = yaml.load(f)

            if doc["info"]["gender"] != "male":
                continue

            data = doc["innings"][0]['1st innings']['deliveries']
            data2 = None
            if len(doc["innings"]) > 1:
                data2 = doc["innings"][1]['2nd innings']['deliveries']

            venue = doc["info"]["venue"]
            runs = 0
            wicket = 0
            balls = 0
            for datum in data:
                for key, value in datum.items():
                    runs += int(value['runs']['total'])
                    if 'extras' not in value:
                        balls += 1
                    elif 'legbyes' in value['extras'] \
                            or 'byes' in value['extras']:
                        balls += 1

                    if 'wicket' in value:
                        wicket += 1

                    venue_ave = get_venue_average(venue)
                    powerplay_balls = 0
                    if balls < 60:
                        powerplay_balls = balls - 60
                    row = [i, balls, runs, wicket, venue_ave, powerplay_balls, total_overs]
                    if len(matrix_1) == 0:
                        matrix_1 = np.array(row)
                    else:
                        matrix_1 = np.vstack([matrix_1, np.array(row)])
            target = runs
            if balls > 295 or wicket == 10:
                choices = ['test', 'train']
                choice = np.random.choice(choices, p=[0.2, 0.8])
                if choice == 'test':
                    if len(matrix_1_test) == 0:
                        matrix_1_test = matrix_1.copy()
                    else:
                        matrix_1_test = np.vstack([matrix_1_test, matrix_1])
                else:
                    if len(matrix_1_train) == 0:
                        matrix_1_train = matrix_1.copy()
                    else:
                        matrix_1_train = np.vstack([matrix_1_train, matrix_1])

        if len(new_matrix) == 0:
            new_matrix = np.array([i, target, balls])
        else:
            new_matrix = np.vstack([new_matrix, np.array([i, target, balls])])

            '''
            runs = 0
            wicket = 0
            balls = 0
            if data2 is not None:
                for datum in data2:
                    for key, value in datum.items():
                        runs += int(value['runs']['total'])
                        if 'extras' not in value:
                            balls += 1
                        elif 'legbyes' in value['extras'] \
                                or 'byes' in value['extras']:
                            balls += 1
                        if 'wicket' in value:
                            wicket += 1
                        venue_ave = get_venue_average(venue)
                        powerplay_balls = 0
                        if balls < 60:
                            powerplay_balls = balls - 60
                        row = [i, balls, runs, wicket, venue_ave, powerplay_balls, target, total_overs]
                        if len(matrix_2) == 0:
                            matrix_2 = np.array(row)
                        else:
                            matrix_2 = np.vstack([matrix_2, np.array(row)])
            if runs >= target or wicket == 10 or balls >= 295:
                choices = ['test', 'train']
                choice = np.random.choice(choices, p=[0.2, 0.8])
                if choice == 'test':
                    if len(matrix_2_test) == 0:
                        matrix_2_test = matrix_2.copy()
                    else:
                        matrix_2_test = np.vstack([matrix_2_test, matrix_2])
                else:
                    if len(matrix_2_train) == 0:
                        matrix_2_train = matrix_2.copy()
                    else:
                        matrix_2_train = np.vstack([matrix_2_train, matrix_2])
            '''
    pd.DataFrame(new_matrix).to_csv("Final_Scores_1st_Innings.csv",sep=',')
    # pd.DataFrame(matrix_1_train).to_csv("Train_1st_Innings.csv", sep=',')
    # pd.DataFrame(matrix_2_train).to_csv("Train_2nd_Innings.csv", sep=',')
    # pd.DataFrame(matrix_1_test).to_csv("Test_1st_Innings.csv", sep=',')
    # pd.DataFrame(matrix_2_test).to_csv("Test_2nd_Innings.csv", sep=',')


def modify_data(fileName):
    df = pd.read_csv(fileName)
    rows = df.as_matrix()
    matrix = []

    for row in rows:
        new_row = np.array(row.copy())
        balls = int(new_row[2])
        non_powerplay_balls = int(new_row[-1])

        if balls < 60:
            poweplay_balls = 60 - balls
            non_powerplay_balls -= poweplay_balls
        else:
            poweplay_balls = 0
        new_row = np.append(new_row[1:-1], [poweplay_balls, non_powerplay_balls])
        if len(matrix) == 0:
            matrix = np.array(new_row)
        else:
            matrix = np.vstack([matrix, new_row])
    pd.DataFrame(matrix).to_csv("second_innings.csv", sep=',')


# ,match_id,balls,runs,wickets,ground_average,pp_balls_left,total_overs
get_data()

1


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27